In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [330]:
df = pd.read_csv("project_data/train_dataset.csv").dropna(subset=["language"]).reset_index()
test = pd.read_csv("project_data/test_dataset.csv")
labels = df.iloc[:,-1]
# df = df.iloc[:, :-1]


In [18]:
zeroR = [2] * test.shape[0]

In [356]:
print(f"The ZeroR accuracy is {1839/3004}")

The ZeroR accuracy is 0.6121837549933422


In [336]:
def OneR(df, col, labels):
    levels = {}

    if df.dtypes[col]=='int64':
        bins = np.histogram(df[col], bins=4)[1]
        values = pd.DataFrame({'var' :   pd.Series(np.digitize(df[col], bins)), 
                               'label' : labels})  

    else:
        if len(df[col].unique())<100:
            values = pd.DataFrame({'var' :  df[col], 'label' : labels})
        else:
            return [] # too computationally heavy if too many categories
    
    for x in values['var'].unique():
        class_ = values[values['var'] == x].label.value_counts().nlargest(1, keep='all')

        if len(class_)==0: 
            print(class_)
        
        if len(class_)>1: levels[x] = class_.sample(n=1)
        levels[x] = class_.index[0]

    pred = []
    for i in range(values.shape[0]):
        pred.append(levels[values['var'][i]])
    return(pred)

In [338]:
pred_list = {}
for i in range(1,df.shape[1]-1):
    col = df.columns[i]
    predictions = OneR(df, col, labels)
    pred_list[col] = predictions

In [350]:
scores = {}
for col in pred_list.keys():
    if len(pred_list[col])>0:
        accuracy = pd.DataFrame({'pred':pred_list[col], 'actual':labels})
        accuracy['correct'] = accuracy.actual==accuracy.pred
        score = sum(accuracy.correct)/len(accuracy.correct)
        scores[col] = score
top_score = max(scores, key=scores.get)

In [355]:
print(f"The OneR is {top_score} with accuracy : {scores[top_score]}")

The OneR is num_voted_users with accuracy : 0.6363636363636364
